In [0]:
from pyspark.sql.functions import*
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,LongType
from pyspark.sql.functions import row_number,rank,dense_rank
from pyspark.sql.window import Window
from pyspark.sql.functions import udf
from pyspark.sql.functions import current_date,date_format,to_date,datediff,months_between,add_months

In [0]:
data=[("HDFC","Chennai",1050010123940000),
      ("CANARA","Bangalore",2050010123940001),
      ("INDIAN","Hyderabad",3050010123940002),
      ("UNION","Mumbai",4050010123940003)]

schema=StructType([
    StructField("Bank",StringType()),
    StructField("Address",StringType()),
    StructField("AccountNumber",LongType())
])

df=spark.createDataFrame(data=data,schema=schema)
display(df)

Bank,Address,AccountNumber
HDFC,Chennai,1050010123940000
CANARA,Bangalore,2050010123940001
INDIAN,Hyderabad,3050010123940002
UNION,Mumbai,4050010123940003


In [0]:
star_df = df.withColumn("AccountNumberStared",concat(lit('*' * 12),substring(col("AccountNumber").cast("string"), -4, 4)))
display(star_df)

Bank,Address,AccountNumber,AccountNumberStared
HDFC,Chennai,1050010123940000,************0000
CANARA,Bangalore,2050010123940001,************0001
INDIAN,Hyderabad,3050010123940002,************0002
UNION,Mumbai,4050010123940003,************0003


In [0]:
data=[("Nancy","HR",2000),
      ("Lishma","HR",3000),
      ("Lav","IT",3000),
      ("sowmi","HR",2500),
      ("joe","IT",3000),
      ("raj","IT",1500),
      ("rayan","IT",4000),
      ("Diya","PayRoll",2500),
      ("Vishnu","PayRoll",2000)]
schema=StructType([
    StructField("Name",StringType()),
    StructField("Dep",StringType()),
    StructField("Salary",IntegerType())
])
df=spark.createDataFrame(data=data,schema=schema)
# df.show()
# df.sort("Dep").show()
window=Window.partitionBy("Dep").orderBy("Salary")
df.withColumn("rownumber",row_number().over(window)).show()
df.withColumn("rank",rank().over(window)).show()
df.withColumn("denserank",dense_rank().over(window)).show()

+------+-------+------+---------+
|  Name|    Dep|Salary|rownumber|
+------+-------+------+---------+
| Nancy|     HR|  2000|        1|
| sowmi|     HR|  2500|        2|
|Lishma|     HR|  3000|        3|
|   raj|     IT|  1500|        1|
|   Lav|     IT|  3000|        2|
|   joe|     IT|  3000|        3|
| rayan|     IT|  4000|        4|
|Vishnu|PayRoll|  2000|        1|
|  Diya|PayRoll|  2500|        2|
+------+-------+------+---------+

+------+-------+------+----+
|  Name|    Dep|Salary|rank|
+------+-------+------+----+
| Nancy|     HR|  2000|   1|
| sowmi|     HR|  2500|   2|
|Lishma|     HR|  3000|   3|
|   raj|     IT|  1500|   1|
|   Lav|     IT|  3000|   2|
|   joe|     IT|  3000|   2|
| rayan|     IT|  4000|   4|
|Vishnu|PayRoll|  2000|   1|
|  Diya|PayRoll|  2500|   2|
+------+-------+------+----+

+------+-------+------+---------+
|  Name|    Dep|Salary|denserank|
+------+-------+------+---------+
| Nancy|     HR|  2000|        1|
| sowmi|     HR|  2500|        2|
|Lishma| 

In [0]:
data = [(1,"Nancy",2000,500),
        (2,"lish",3000,1000)]
schema = StructType([
    StructField("ID",IntegerType()),
    StructField("Name",StringType()),
    StructField("Salary",IntegerType()),
    StructField("Bonus",IntegerType())    
])
df=spark.createDataFrame(data=data,schema=schema)
df.display()

ID,Name,Salary,Bonus
1,Nancy,2000,500
2,lish,3000,1000


In [0]:
def totalpay(s,b):
    return s+b 
# help(udf)
TotalPayment = udf(lambda s,b : totalpay(s,b),IntegerType())
df.withColumn("totalpay",TotalPayment(df.Salary,df.Bonus)).show()

+---+-----+------+-----+--------+
| ID| Name|Salary|Bonus|totalpay|
+---+-----+------+-----+--------+
|  1|Nancy|  2000|  500|    2500|
|  2| lish|  3000| 1000|    4000|
+---+-----+------+-----+--------+



In [0]:
data = [(1,"Nancy",2000,500),
        (2,"lish",3000,1000)]
schema = StructType([
    StructField("ID",IntegerType()),
    StructField("Name",StringType()),
    StructField("Salary",IntegerType()),
    StructField("Bonus",IntegerType())    
])
df=spark.createDataFrame(data,schema)
df.createOrReplaceTempView('emp')

In [0]:
%sql
select * from emp

ID,Name,Salary,Bonus
1,Nancy,2000,500
2,lish,3000,1000


In [0]:
df=spark.range(2)
df1 = df.withColumn("currentdate",current_date())
df1.show()
df2=df1.withColumn("currentdate",to_date(df1.currentdate,'yyyy-dd-mm'))
df2.show()

+---+-----------+
| id|currentdate|
+---+-----------+
|  0| 2023-12-20|
|  1| 2023-12-20|
+---+-----------+

+---+-----------+
| id|currentdate|
+---+-----------+
|  0| 2023-12-20|
|  1| 2023-12-20|
+---+-----------+



In [0]:
df=spark.createDataFrame([('2023-11-20','2023-12-20')], ['d1' ,'d2'])
df.show()
df.withColumn("datediff",datediff(df.d2,df.d1)).show()
df.withColumn('monthsbetween',months_between(df.d2,df.d1)).show()
df.withColumn('addmonths',add_months(df.d2 , 10)).show()
df.withColumn('submonth',add_months(df.d2 , -3)).show()

+----------+----------+
|        d1|        d2|
+----------+----------+
|2023-11-20|2023-12-20|
+----------+----------+

+----------+----------+--------+
|        d1|        d2|datediff|
+----------+----------+--------+
|2023-11-20|2023-12-20|      30|
+----------+----------+--------+

+----------+----------+-------------+
|        d1|        d2|monthsbetween|
+----------+----------+-------------+
|2023-11-20|2023-12-20|          1.0|
+----------+----------+-------------+

+----------+----------+----------+
|        d1|        d2| addmonths|
+----------+----------+----------+
|2023-11-20|2023-12-20|2024-10-20|
+----------+----------+----------+

+----------+----------+----------+
|        d1|        d2|  submonth|
+----------+----------+----------+
|2023-11-20|2023-12-20|2023-09-20|
+----------+----------+----------+

